In [1]:
"""
topic modeling of r/technews posts
"""

'\ntopic modeling of r/technews posts\n'

In [2]:
import pandas as pd
import numpy as np
import pickle
import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [6]:
with open('preprocessed_posts.pickle', 'rb') as read_file:
    posts = pickle.load(read_file)
    
posts.shape

(48082, 18)

In [7]:
posts.head()

,id,created_utc,date_time,author,author_fullname,title,url,domain,num_comments,score,permalink,full_link,refine_title,preproc_title,unstemmed,stem_title,processed_title,title_nouns
0,29j6ov,1404184792,2014-07-01 03:19:52,Password_is_123456,t2_6crtu,New leaks show off the iPhone 6's curved edged...,http://www.cnet.com/news/new-leaks-show-off-up...,cnet.com,0,0,/r/technews/comments/29j6ov/new_leaks_show_off...,https://www.reddit.com/r/technews/comments/29j...,New leaks show off the iPhone 6's curved edged...,New leaks show off the iPhone s curved edged...,"[New, leaks, show, off, the, iPhone, s, curved...","[new, leak, show, off, the, iphon, s, curv, ed...",new leak show off the iphon s curv edg display,New leaks iPhone s display
1,29j6op,1404184791,2014-07-01 03:19:51,Password_is_123456,t2_6crtu,Millions of dynamic DNS users suffer after Mic...,http://arstechnica.com/security/2014/06/millio...,arstechnica.com,6,63,/r/technews/comments/29j6op/millions_of_dynami...,https://www.reddit.com/r/technews/comments/29j...,Millions of dynamic DNS users suffer after Mic...,Millions of dynamic DNS users suffer after Mic...,"[Millions, of, dynamic, DNS, users, suffer, af...","[million, of, dynam, dns, user, suffer, after,...",million of dynam dns user suffer after microso...,Millions DNS users Microsoft IP domains
2,29j2c1,1404181722,2014-07-01 02:28:42,[deleted],NaN,Paypal Freezes ProtonMail Campaign Funds,https://protonmail.ch/blog/paypal-freezes-prot...,protonmail.ch,0,1,/r/technews/comments/29j2c1/paypal_freezes_pro...,https://www.reddit.com/r/technews/comments/29j...,Paypal Freezes ProtonMail Campaign Funds,Paypal Freezes ProtonMail Campaign Funds,"[Paypal, Freezes, ProtonMail, Campaign, Funds]","[paypal, freez, protonmail, campaign, fund]",paypal freez protonmail campaign fund,Paypal Freezes ProtonMail Campaign Funds
3,29hy6f,1404156644,2014-06-30 19:30:44,ANIMAL_NewYork,t2_h1mjr,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,http://animalnewyork.com/2014/facebook-emotion...,animalnewyork.com,0,1,/r/technews/comments/29hy6f/unethical_facebook...,https://www.reddit.com/r/technews/comments/29h...,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,"[UNETHICAL, FACEBOOK, “EMOTIONAL, CONTAGION”, ...","[uneth, facebook, “emot, contagion”, studi, no...",uneth facebook “emot contagion” studi not fund...,FACEBOOK “ EMOTIONAL CONTAGION ” STUDY NOT FUN...
4,29hvyr,1404155395,2014-06-30 19:09:55,Password_is_123456,t2_6crtu,Microsoft May Prioritize The Desktop In Windows 9,http://techcrunch.com/2014/06/30/microsoft-may...,techcrunch.com,23,53,/r/technews/comments/29hvyr/microsoft_may_prio...,https://www.reddit.com/r/technews/comments/29h...,Microsoft May Prioritize The Desktop In Windows 9,Microsoft May Prioritize The Desktop In Windows,"[Microsoft, May, Prioritize, The, Desktop, In,...","[microsoft, may, priorit, the, desktop, in, wi...",microsoft may priorit the desktop in window,Microsoft May Desktop Windows


In [13]:
# add to list of stop words

stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['new', 'news', 'tech', '&amp;', 'amp', 'million', 'billion'] )


In [14]:
### initial topic model

In [15]:
vectorizer = CountVectorizer(stop_words = stopwords, ngram_range=(1,2))
title_words = vectorizer.fit_transform(posts.processed_title)
title_words.shape

(48082, 233646)

In [17]:
nmf_model = NMF(25)  # tried 5, 7, 10, 12, 15, 20, 25, 30
post_topic = nmf_model.fit_transform(title_words)

words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

/Users/amybutler/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


[['googl', 'search', 'pixel', 'chrome', 'googl pixel', 'play'],
 ['appl', 'watch', 'io', 'appl watch', 'macbook', 'store'],
 ['samsung', 'galaxi', 'samsung galaxi', 'note', 'galaxi note', 'leak'],
 ['pro', 'camera', 'huawei', 'xiaomi', 'smartphon', 'redmi'],
 ['app', 'mobil', 'develop', 'store', 'app develop', 'app store'],
 ['facebook', 'user', 'video', 'ad', 'messeng', 'twitter'],
 ['launch', 'india', 'price', 'spacex', 'launch india', 'watch'],
 ['microsoft', 'window', 'free', 'updat', 'pc', 'download'],
 ['servic', 'develop', 'develop servic', 'test', 'softwar', 'mobil'],
 ['phone', 'mobil', 'phone number', 'mobil phone', 'number', 'android phone'],
 ['use', 'ai', 'comput', 'use ai', 'system', 'machin'],
 ['data', 'user', 'breach', 'data breach', 'leak', 'center'],
 ['support', 'technic', 'technic support', 'number', 'help', 'window'],
 ['get', 'updat', 'free', 'help', 'get free', 'featur'],
 ['iphon', 'plus', 'appl iphon', 'iphon plus', 'featur', 'iphon iphon'],
 ['secur', 'networ

In [ ]:
# with 20 NMF above, ai in two topics
# still in ai in 2 topics with 25 topics

In [18]:
## try topic modeling with title_nouns

vectorizer = CountVectorizer(stop_words = stopwords, ngram_range=(1,2))
title_words = vectorizer.fit_transform(posts.title_nouns)
title_words.shape

nmf_model = NMF(20)  # tried 5, 7, 10, 12, 15, 17, 20, 25, 27, 30
post_topic = nmf_model.fit_transform(title_words)

words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

/Users/amybutler/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


[['google', 'pixel', 'search', 'chrome', 'google pixel', 'play'],
 ['apple', 'watch', 'apple watch', 'macbook', 'ios', 'pro'],
 ['samsung', 'galaxy', 'samsung galaxy', 'note', 'galaxy note', 'camera'],
 ['facebook', 'users', 'messenger', 'twitter', 'zuckerberg', 'instagram'],
 ['windows', 'pc', 'download', 'mac', 'support', 'windows mac'],
 ['android', 'apps', 'update', 'android app', 'phones', 'pie'],
 ['data', 'breach', 'data breach', 'users', 'center', 'data center'],
 ['number', 'support', 'customer', 'email', 'care', 'customer care'],
 ['app', 'development', 'mobile', 'store', 'services', 'app development'],
 ['china', 'us', 'huawei', 'companies', 'trump', 'trade'],
 ['security',
  'services',
  'network',
  'testing',
  'software',
  'testing services'],
 ['iphone', 'apple iphone', 'iphone iphone', 'xs', 'se', 'iphone se'],
 ['amazon', 'prime', 'echo', 'alexa', 'tv', 'amazon echo'],
 ['microsoft',
  'surface',
  'xbox',
  'microsoft surface',
  'microsoft windows',
  'office'],
 

In [ ]:
# 20 topics got ai in a separate topic, tesla in it's own topic
# most of these 20 can be easily labeled
# a couple coule be combined

In [19]:
posts['post_topic_nmf'] = post_topic.argmax(axis=1)

In [20]:
# the CountVectorizer + NMF yielded pretty seaparate and easily labeled topics
# with AI and machine learning separate

with open('topics_posts.pickle', 'wb') as to_write:
    pickle.dump(posts, to_write)

In [ ]:
####################

In [ ]:
# CountVectorizer + LDA

In [24]:
vectorizer = CountVectorizer(stop_words = stopwords, ngram_range=(1,2))
title_words = vectorizer.fit_transform(posts.processed_title)

lda_model = LatentDirichletAllocation(n_components=20)
doc_topic = lda_model.fit_transform(title_words)
doc_topic.shape

(48082, 20)

In [25]:
words = vectorizer.get_feature_names()
t = lda_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['camera', 'launch', 'xiaomi', 'pro', 'googl', 'price'],
 ['googl', 'appl', 'use', 'watch', 'android', 'say'],
 ['app', 'develop', 'appl', 'amazon', 'android', 'servic'],
 ['musk', 'elon', 'elon musk', 'smart', 'ai', 'facebook'],
 ['phone', 'support', 'number', 'make', 'hp', 'iphon'],
 ['window', 'download', 'technolog', 'free', 'support', 'mac'],
 ['googl', 'test', 'servic', 'data', 'facebook', 'help'],
 ['social', 'media', 'social media', 'network', 'market', 'secur'],
 ['appl', 'io', 'app', 'best', 'mobil', 'iphon'],
 ['googl', 'self', 'drive', 'self drive', 'car', 'appl'],
 ['silicon', 'secur', 'valley', 'silicon valley', 'servic', 'first'],
 ['oneplus', 'appl', 'get', 'launch', 'us', 'phone'],
 ['net', 'neutral', 'net neutral', 'googl', 'appl', 'app'],
 ['googl', 'first', 'launch', 'support', 'ai', 'facebook'],
 ['facebook', 'know', 'updat', 'fi', 'xbox', 'secur'],
 ['window', 'microsoft', 'app', 'android', 'googl', 'internet'],
 ['deal', 'number', 'iphon', 'custom', 'care', 'ema

In [ ]:
# google in almost every topic with title_nouns input
# with processed_title still has some key words in multiple topics

In [ ]:
###################

In [ ]:
### try tfidf vectorizer

In [29]:
# create the document-term matrix with TF-IDF vectorizer 
cv_tfidf = TfidfVectorizer(stop_words=stopwords)
X_tfidf = cv_tfidf.fit_transform(posts.title_nouns).toarray()


In [27]:
nmf_model = NMF(27)  
post_topic2 = nmf_model.fit_transform(X_tfidf)
post_topic2.shape

/Users/amybutler/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


(48082, 27)

In [28]:
words2 = vectorizer.get_feature_names()
t2 = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t2]
topic_words

[['anoth front',
  'bad processor',
  'begin whi',
  'ago reinvent',
  'acquir cumulus',
  'bag fund'],
 ['bee behaviour',
  'announc handheld',
  'automat tune',
  'andromeda multiplay',
  'beach virginia',
  'bluewhal technolog'],
 ['account twitter',
  'boost sector',
  'appl last',
  'appl level',
  'asahi',
  'blockchain regul'],
 ['analysi explain',
  'asus pronounc',
  'abstract platform',
  'armour provid',
  'braucht',
  'appl cisco'],
 ['bother big',
  'back huge',
  'aryz',
  'altern appl',
  'bluewhal technolog',
  'android notif'],
 ['accord us',
  'accur recogn',
  'bluewhal technolog',
  'backwash',
  'allow open',
  'ad place'],
 ['appl let',
  'begin today',
  'accur recogn',
  'anchor deliv',
  'begin leverag',
  'appl lay'],
 ['backlash facial',
  'attack code',
  'autonom truck',
  'afghan femal',
  'bad processor',
  'begin leverag'],
 ['bola processor',
  'branch',
  'brain product',
  'appl cisco',
  'boss financ',
  'announc live'],
 ['belat',
  'allow manufactu